In [1]:
import os
from datetime import date, timedelta

import mariadb
import pandas as pd
from dotenv import load_dotenv
from pandas.tseries.holiday import USFederalHolidayCalendar

_ = load_dotenv("config.env")

DATABASE_NAME: str = "data_mart"

In [2]:
connection = mariadb.connect(
    host="127.0.0.1",
    port=23306,
    user=os.getenv("user"),
    password=os.getenv("password"),
    autocommit=True,
)
cursor = connection.cursor()

In [3]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS {db}.stores (
        store_key SMALLINT PRIMARY KEY NOT NULL,
        store_manager TINYTEXT NOT NULL,
        street_address TINYTEXT NOT NULL,
        city TINYTEXT NOT NULL,
        state TINYTEXT NOT NULL,
        zip_code TINYTEXT NOT NULL,
        phone_number TINYTEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

In [4]:
cursor.execute(
    "INSERT IGNORE INTO {db}.stores (store_key, store_manager, street_address, city, state, zip_code, phone_number) VALUES (?, ?, ?, ?, ?, ?, ?)".format(
        db=DATABASE_NAME
    ),
    (
        8,
        "Jason Snouffer",
        "1640 Riverside Drive",
        "Hill Valley",
        "CA",
        "91905",
        "831-555-4385",
    ),
)

In [5]:
cursor.execute("DROP TABLE IF EXISTS {db}.dates".format(db=DATABASE_NAME))
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS {db}.dates (
        date_key SMALLINT PRIMARY KEY NOT NULL,
        datetime DATETIME NOT NULL,
        day_in_month SMALLINT NOT NULL,
        day_in_year SMALLINT NOT NULL,
        week_number SMALLINT  NOT NULL,
        month_number SMALLINT NOT NULL,
        month_text TINYTEXT NOT NULL,
        quarter SMALLINT NOT NULL,
        year SMALLINT NOT NULL,
        fiscal_year SMALLINT NOT NULL,
        is_holiday BOOLEAN NOT NULL,
        is_weekend BOOLEAN NOT NULL,
        season TINYTEXT NOT NULL
    )
    """.format(
        db=DATABASE_NAME
    )
)

In [6]:
year = 2021
current_date: date = date(year, 1, 1)
stop_date: date = date(year, 12, 31)
fiscal_year_start = date(year, 8, 1)

holidays = USFederalHolidayCalendar().holidays(start=current_date.isoformat(), end=stop_date.isoformat()).to_pydatetime()
holidays = [d.date() for d in holidays]

seasons = [('winter', (date(year,  1,  1),  date(year,  3, 20))),
           ('spring', (date(year,  3, 21),  date(year,  6, 20))),
           ('summer', (date(year,  6, 21),  date(year,  9, 22))),
           ('autumn', (date(year,  9, 23),  date(year, 12, 20))),
           ('winter', (date(year, 12, 21),  date(year, 12, 31)))]

while current_date <= stop_date:   
    time_tuple = current_date.timetuple()
    weekday = current_date.strftime("%A")
    
    cursor.execute(
        """
        INSERT IGNORE INTO {db}.dates (
            date_key, datetime, day_in_month, day_in_year, week_number, month_number, month_text,
            quarter, year, fiscal_year, is_holiday, is_weekend, season
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """.format(
            db=DATABASE_NAME
        ),
        (
            time_tuple.tm_yday,
            current_date.isoformat(),
            time_tuple.tm_mday,
            time_tuple.tm_yday,
            current_date.isocalendar()[1],
            time_tuple.tm_mon,
            current_date.strftime("%B"),
            pd.Timestamp(current_date).quarter,
            time_tuple.tm_year,
            time_tuple.tm_year if current_date >= fiscal_year_start else time_tuple.tm_year - 1,
            True if current_date in holidays else False,
            True if weekday == "Saturday" or weekday == "Sunday" else False,
            next(season for season, (start, end) in seasons if start <= current_date <= end)
        ),
    )

    current_date = current_date + timedelta(days=1)